<a href="https://colab.research.google.com/github/shiffa-04/Titanic-ML-Model/blob/main/T_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import regularizers

In [ ]:
df = pd.read_csv("modified_file.csv")
columns_to_drop = ['Survived','PassengerId', 'Ticket', 'Fare', 'Name']
X = df.drop(columns = columns_to_drop)
y= df['Survived']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
   transformers = [('num',numerical_transformer,['Age','SibSp', 'Parch'] ),
                   ('cat', categorical_transformer,['Pclass', 'Sex','Embarked'])
   ])

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state = 10)

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

X_train_shape = X_train.shape
X_test_shape = X_test.shape

print(f'Shape of X_train :{X_train_shape}')
print(f'Shape of X_test:{X_test_shape}')

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



Shape of X_train :(712, 12)
Shape of X_test:(179, 12)


In [ ]:
model = Sequential(
    [
        Dense(units = 64, input_dim = X_train.shape[1], kernel_regularizer=regularizers.l2(0.01), activation = 'relu', name='l1'),
        Dropout(rate=0.3, name='dropout1'),
        Dense(units = 1, activation = 'sigmoid', name='l3')
    ]
)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
# batch_size = 20
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
23/23 [==============================] - 1s 11ms/step - loss: 0.8427 - accuracy: 0.6404 - val_loss: 0.7536 - val_accuracy: 0.7598
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.7651 - accuracy: 0.7037 - val_loss: 0.6889 - val_accuracy: 0.8045
Epoch 3/100
23/23 [==============================] - 0s 6ms/step - loss: 0.7073 - accuracy: 0.7514 - val_loss: 0.6385 - val_accuracy: 0.8212
Epoch 4/100
23/23 [==============================] - 0s 8ms/step - loss: 0.6658 - accuracy: 0.7753 - val_loss: 0.5987 - val_accuracy: 0.8268
Epoch 5/100
23/23 [==============================] - 0s 6ms/step - loss: 0.6281 - accuracy: 0.7935 - val_loss: 0.5692 - val_accuracy: 0.8212
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5984 - accuracy: 0.7978 - val_loss: 0.5441 - val_accuracy: 0.8268
Epoch 7/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5781 - accuracy: 0.8076 - val_loss: 0.5290 - val_accuracy: 0.8212
Epoch 8/100


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100 :.2f}')

6/6 [==============================] - 0s 4ms/step - loss: 0.4006 - accuracy: 0.8547
Test Accuracy: 85.47
